In [ ]:
# selenium 불러오기
from selenium import webdriver
import time

# chrome창(웹드라이버) 열기
driver = webdriver.Chrome("./chromedriver")

# 실행할 웹페이지 불러오기(구글 지도 예시)
driver.get("https://www.google.com/maps/")

# 선택자로 페이지 요소 선택하기(단일)
driver.find_element_by_css_selector("선택자")

# 선택자로 페이지 요소 선택하기(다중)
driver.find_elements_by_css_selector("선택자")

# 해당 요소에 문자 입력하기(ex. 버튼)
button.send_keys("텍스트")

# 해당 요소 클릭하기
button.click()

# 의도적으로 3초 쉬어주기
time.sleep(3)

# 여러 페이지(999)에서 반복하기
for i in range(999):
    # 시간 지연
    time.sleep(3)

    # 컨테이너(가게) 데이터 수집 // div.section-result-content
    stores = driver.find_elements_by_css_selector("div.section-result-content")

    for s in stores:
        # 가게 이름 데이터 수집 // h3.section-result-title
        title = s.find_element_by_css_selector("h3.section-result-title").text

        # 평점 데이터 수집 // span.cards-rating-score
        # 평점이 없는 경우 에러 처리
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"

        # 가게 주소 데이터 수집 // span.section-result-location
        addr = s.find_element_by_css_selector("span.section-result-location").text

        print(title, "/", score, "/", addr)

    # 다음페이지 버튼 클릭 하기
    # 다음페이지가 없는 경우(데이터 수집 완료) 에러 처리
    try:
        nextpage = driver.find_element_by_css_selector("button#n7lv7yjyC35__section-pagination-button-next")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import pymysql

query = '미용실' #검색어
sortingOrder = 'reviewCount' #정렬방식
# lat = '35.179949125560846' #위도 y
# lng = '129.07495481148936' #경도 x
# location = "부산광역시청"

arr_location=[
    '서산시청',
    '논산시청',
    '계룡시청',
    '당진시청',
    '금산군청',
    '부여군청',
    '서천군청',
    '청양군청',
    '홍성군청',
    '예산군청',
    '태안군청',
    '청주시청',
    '충주시청',
    '제천시청',
    '보은군청',
    '옥천군청',
    '영동군청',
    '진천군청',
    '괴산군청',
    '음성군청',
    '단양군청',
    '증평군청',
    '춘천시청',
    '원주시청',
    '강릉시청',
    '동해시청',
    '태백시청',
    '속초시청',
    '삼척시청',
    '홍천군청',
    '횡성군청',
    '영월군청',
    '평창군청',
    '정선군청',
    '철원군청',
    '화천군청',
    '양구군청',
    '인제군청',
    '강원고성군청',
    '양양군청',
    '수원시청',
    '성남시청',
    '고양시청',
    '용인시청',
    '부천시청',
    '안산시청',
    '안양시청',
    '남양주시청',
    '화성시청',
    '평택시청',
    '의정부시청',
    '시흥시청',
    '파주시청', 
    '광명시청',
    '김포시청',
    '군포시청', 
    '광주시청', 
    '이천시청', 
    '오산시청', 
    '구리시청',
    '안성시청', 
    '포천시청', 
    '의왕시청', 
    '하남시청', 
    '여주시청', 
    '양평군청', 
    '동두천시청',
    '과천시청',
    '가평군청', 
    '연천군청',
    '제주도청',
    '세종특별자치시청',
    '울산광역시청',
    '대전광역시청',
    '광주광역시청',
    '인천광역시청',
    '대구광역시청',
    '서울특별시청'
]

arr_lat = [
    '36.78503869',
    '36.18739278',
    '36.27440217',
    '36.89252967',
    '36.10905587',
    '36.27587367',
    '36.08056045',
    '36.45932113',
    '36.60152612',
    '36.67997098',
    '36.74574993',
    '36.64310836',
    '36.99113745',
    '37.13277026',
    '36.48955917',
    '36.30694699',
    '36.17519669',
    '36.85506642',
    '36.81552414',
    '36.94039791',
    '36.98480447',
    '36.78555656',
    '37.88146258',
    '37.34242328',
    '37.75280793',
    '37.52485363',
    '37.16422843',
    '38.20718589',
    '37.45012438',
    '37.69718829',
    '37.49188996',
    '37.18385449',
    '37.37089496',
    '37.38075289',
    '38.14685207',
    '38.10639453',
    '38.11012535',
    '38.06979144',
    '38.38078938',
    '38.07557789',
    '37.26627642',
    '37.42280244',
    '37.65884948',
    '37.24123269',
    '37.50479712',
    '37.32202624',
    '37.39445543',
    '37.63650867',
    '37.19963872',
    '36.99274426',
    '37.73827564',
    '37.38042369',
    '37.76105317',
    '37.47869258',
    '37.61549326',
    '37.36187484',
    '37.42973673',
    '37.27243341',
    '37.14999036',
    '37.59440339',
    '37.01043387',
    '37.89503524',
    '37.34481423',
    '37.53943928',
    '37.29969968',
    '37.49190832',
    '37.90378992',
    '37.42938424',
    '37.83140741',
    '38.09655272',
    '33.49068524',
    '36.48035617',
    '35.54130573',
    '36.35070727',
    '35.16032218',
    '37.45626853',
    '35.87161323',
    '37.56679555'
]

arr_lng = [
    '126.4502878',
    '127.0989118',
    '127.2488377',
    '126.6460284',
    '127.4880854',
    '126.9097354',
    '126.6917407',
    '126.8023025',
    '126.6608962',
    '126.8452256',
    '126.2980581',
    '127.4888331',
    '127.9258304',
    '128.1909693',
    '127.7295259',
    '127.5713603',
    '127.7834305',
    '127.4358332',
    '127.7866624',
    '127.6904693',
    '128.3654309',
    '127.5815101',
    '127.7300226',
    '127.9197831',
    '128.8759508',
    '129.1142488',
    '128.9855543',
    '128.5918088',
    '129.1654442',
    '127.8888322',
    '127.9850144',
    '128.4618154',
    '128.3903487',
    '128.6608674',
    '127.3132959',
    '127.7080664',
    '127.9900668',
    '128.1703563',
    '128.4678275',
    '128.6191494',
    '127.0281814',
    '127.1253981',
    '126.8321999',
    '127.1779422',
    '126.7636163',
    '126.8308689',
    '126.9568552',
    '127.2165892',
    '126.8316522',
    '127.1126252',
    '127.0337545',
    '126.8027838',
    '126.7800154',
    '126.8646628',
    '126.7154188',
    '126.9350988',
    '127.2552573',
    '127.4350552',
    '127.0774069',
    '127.1297531',
    '127.2793461',
    '127.2003611',
    '126.9683208',
    '127.2149687',
    '127.6368865',
    '127.4874168',
    '127.0604392',
    '126.9876194',
    '127.5095764',
    '127.0750459',
    '126.5009412',
    '127.2887863',
    '129.3116504',
    '127.3847758',
    '126.8515509',
    '126.7058406',
    '128.6016857',
    '126.9778297'
]

for i in range(len(arr_location)):
    
    location = arr_location[i]
    lng = arr_lng[i]
    lat = arr_lat[i]
    
    link = 'https://m.place.naver.com/hairshop/list?sm=mtp_hty.top&where=m&query='
    link += query + '&x=' + lng + '&y=' + lat + '&level=top&entry=pll&sortingOrder=' + sortingOrder
    link += '&gender=f&listViewQuery=' + query

    # 크롬창(웹드라이버) 열기
    driver = webdriver.Chrome("./chromedriver")

    # 네이버 플레이스 링크 접속
    driver.get(link)
    time.sleep(3)

    last_height = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2)").size
    try:
        hover = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(1) > div._15UxR > div > div > a._2QhH-._1-QNb")
        hover.send_keys(Keys.ENTER)
    except:
        hover = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(1) > div > div._1ku_V._2ys5d > div > a:nth-child(1)")
        hover.send_keys(Keys.ENTER)

    while True:
        for i in range(30):
            hover.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        size = driver.find_element_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2)").size
        if size['height'] == last_height['height']:
            time.sleep(10)
            if size['height'] == last_height['height']:
                print("데이터 로드 완료")
                break
            print('네트워크 쫌 느림 한번 더 실행')
        last_height = size
    #     print(size['height'])

    stores = driver.find_elements_by_css_selector("#_list_scroll_container > div > div > div:nth-child(2) > ul > li > div._1WDkQ > div._1PT3q")
    arrTitle = []
    arrHref = []
    for s in stores:
        title = s.find_element_by_css_selector("span._3t53y").text
        href = s.find_element_by_css_selector("a._1ptue").get_attribute('href')
        href = href.partition('hairshop/')[2].split('?')[0]
        arrTitle.append(title)
        arrHref.append(href)
    # print(arrTitle)
    # print(arrHref)

    conn = pymysql.connect(
        charset='utf8',
        user='root', 
        passwd='', 
        host='localhost',
        db='test'
    )

    cursor = conn.cursor()

    # # 테이블생성
    # sql = '''CREATE TABLE hairshop ( 
    # c_idx int(11) NOT NULL AUTO_INCREMENT PRIMARY KEY, 
    # c_idx_naver int(20),
    # c_name varchar(255),
    # c_search_location varchar(50)
    # ) 
    # ''' 
    # cursor.execute(sql)

    # SELECT
    # sql = "SELECT * FROM `hair_commerce_tb`;"
    # cursor.execute(sql)
    # result = cursor.fetchall()
    # print(result)

    for title,href in zip(arrTitle, arrHref):
        sql = "INSERT INTO hairshop (c_name, c_idx_naver, c_search_location) VALUES (%s,%s,%s)"
        val = (title,href,location)
        cursor.execute(sql, val)

    conn.commit()
    conn.close()
    print("'{0}' 데이터 저장 완료".format(location))

    # 크롬창 닫기
    driver.close()

In [19]:
import requests
import time
import pymysql
import re
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from fake_useragent import UserAgent

ua = UserAgent()
# ua = UserAgent(verify_ssl=False)
headers = {'User-Agent' : ua.random}

conn = pymysql.connect(
    charset='utf8',
    user='root', 
    passwd='', 
    host='localhost',
    db='test'
)
cursor = conn.cursor()

sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '부산광역시청' AND `c_idx` > 1205;"
# sql = "SELECT * FROM `hairshop` WHERE `c_search_location` = '창원시청' AND `c_idx` = 670;"
cursor.execute(sql)
result = cursor.fetchall()
# print(result[0][0]) #c_idx
# print(result[0][1]) #c_idx_naver
# print(result[0][2]) #c_name
# print(result[0][3]) #c_search_location
# print(len(result))

for res in result:
    c_home = []
    c_price = []
    c_designer = []
    isDesigner = 0
    isPrice = 0

    #----------------------------home---------------------------
    url = 'https://m.place.naver.com/hairshop/{0}/home?entry=pll'.format(res[1])
    response = requests.get(url, headers=headers)
    
    if response.status_code == 500:
        time.sleep(0.5)

    if response.status_code == 200:
        html = response.content.decode('utf-8','replace')
        soup = BeautifulSoup(html, 'html.parser')
        try:
            c_phone = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div.afDDM > div > span:nth-child(1) > a').attrs['href'].split(':')[1]
        except:
            print("{0}의 c_phone 못찾음".format(url))
            c_phone = -1
        try:
            c_addr = soup.select_one('li._1M_Iz._1aj6- > div > span._2yqUQ').text
        except:
            print("{0}의 c_addr 못찾음".format(url))
            c_addr = -1
        try:
            c_hours = soup.select_one('li._1M_Iz._2KHqk > div > div > div').text
        except:
            try:
                c_hours = soup.select_one('li._1M_Iz._2KHqk > div > a > div:nth-child(1) > div > span').text
            except:
                print("{0}의 c_hours 못찾음".format(url))
                c_hours = -1
        try:
            c_score = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span._1Y6hi._1A8_M > em').text
        except:
            print("{0}의 c_score 못찾음".format(url))
            c_score = -1
        try:
            c_visitor_review = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(1) > a > em').text
        except:
            try:
                c_visitor_review = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3XpyR > div > span:nth-child(2) > a > em').text
            except:
                print("{0}의 c_visitor_review 못찾음".format(url))
                c_visitor_review = -1
        try:
            c_is_reservation = soup.select_one('#app-root > div > div.place_detail_wrapper > div.place_section.no_margin.GCwOh > div > div > div._3wP2G > div > span')
            if c_is_reservation == None:
                c_is_reservation = -1
            else:
                c_is_reservation = 1
        except:
            c_is_reservation = -1
        try:
            element = soup.select('#app-root > div > div.place_detail_wrapper > div.place_fixed_maintab > div > div > div > div > a')
            for ele in element:
                e = ele.select_one('span._3aXen').text
                if e == '스타일리스트':
                    isDesigner = 1
                if e == '가격':
                    isPrice = 1
        except:
            pass
                    
        c_home.append((c_phone, c_addr, c_hours, c_score, c_visitor_review, c_is_reservation))
        time.sleep(0.5)
    else : 
        print("페이지 로드 실패 : {0}".format(url))
        print(response.status_code)

    # ----------------------------price---------------------------
    if isPrice == 1:
        url = 'https://m.place.naver.com/hairshop/{0}/price?entry=pll'.format(res[1])
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            html = response.content.decode('utf-8','replace')
            soup = BeautifulSoup(html, 'html.parser')

            try:
                element = soup.select('#app-root > div > div.place_detail_wrapper > div:nth-child(4) > div.place_section.no_margin > div > div > ul > li')
                for ele in element:
                    try:
                        kind = ele.select_one('span._1kuzz').string
                    except:
                        print("{0}의 kind 못찾음".format(url))
                        kind = -1
                    ele_li = ele.select('ul._12NKi > li')
                    for li in ele_li:
                        try:
                            name = li.select_one('div > div > span > span').string
                        except:
                            print("{0}의 name 못찾음".format(url))
                            name = -1
                        try:
                            price = re.sub(",","",li.select_one('em._12eac').text[0:-1])
                        except:
                            print("{0}의 price 못찾음".format(url))
                            price = -1
                        c_price.append((res[1], kind, name, price))
            except:
                print("price의 element or ele_li 못 찾음 - {0}".format(url))
            time.sleep(0.5)
        else : 
            print("페이지 로드 실패 : {0}".format(url))
            print(response.status_code)

    #----------------------------stylist---------------------------
    if isDesigner == 1:
        
        url = 'https://m.place.naver.com/hairshop/{0}/stylist'.format(res[1])
        driver = webdriver.Chrome("./chromedriver")
        driver.get(url)
        time.sleep(2.5)

        try:
            more = driver.find_element_by_css_selector("#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div > div.place_section._3a4xA > div._2kAri > a")
            more.click()
            time.sleep(0.5)
        except:
            pass

        designers = driver.find_elements_by_css_selector('#app-root > div > div.place_detail_wrapper > div:nth-child(5) > div > div.place_section._3a4xA > div:nth-child(2) > ul > li')
        for designer in designers:
            try:
                d_idx_naver = designer.find_element_by_css_selector('div._2TEwD > div.NacXe > a').get_attribute('href').partition('stylist/')[2]
            except:
                print("{0}의 d_idx_naver 못찾음".format(url))
                d_idx_naver = -1
            try:
                d_name = designer.find_element_by_css_selector('div._2TEwD > div.NacXe > a > span').text
            except:
                print("{0}의 d_name 못찾음".format(url))
                d_name = -1
            try:
                d_comment = designer.find_element_by_css_selector('div._2TEwD > div._2KAFb > a').text
            except:
                print("{0}의 d_comment 못찾음".format(url))
                d_comment = -1
            try:
                d_score = designer.find_element_by_css_selector('div._2TEwD > div._1Ezoe > a > span._3XrWs > em').text
            except:
                print("{0}의 d_score 못찾음".format(url))
                d_score = -1
            try:
                d_reservation_link = designer.find_element_by_css_selector('div.OO9oN > a').get_attribute('href')
            except:
                print("{0}의 d_reservation_link 못찾음".format(url))
                d_reservation_link = -1
            try:
                d_img_src = designer.find_element_by_css_selector('div.Deo8K > a > img').get_attribute('src')
            except:
                print("{0}의 d_img_src 못찾음".format(url))
                d_img_src = -1
            c_designer.append((d_idx_naver, res[1], d_name, d_score, d_reservation_link, d_img_src, d_comment))

        driver.close()


    # ----------------------------db저장---------------------------
#     print(c_home)
#     print(c_price)
#     print(c_designer)
#     print("'{0}'매장 데이터 저장완료".format(res[2]))
#     print("--------------------------------------------------------------------------------------")
    
    try:
        sql = "UPDATE hairshop SET c_phone=%s, c_addr=%s, c_hours=%s, c_score=%s, edit_date=%s, c_visitor_review=%s, c_is_reservation=%s WHERE c_idx=%s"
        val = (c_home[0][0], c_home[0][1], c_home[0][2], c_home[0][3], datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), c_home[0][4], c_home[0][5], res[0])
        cursor.execute(sql, val)
    except:
        print("sql-Update Hairshop fail : {0}".format(val))
    
    if isPrice == 1:
        try:
            for (c_idx_naver, kind, name, price) in c_price:
                sql = "INSERT INTO price (c_idx_naver, p_kind, p_name, p_price) VALUES (%s,%s,%s,%s)"
                val = (c_idx_naver, kind, name, price)
                cursor.execute(sql, val)
        except:
            print("sql-Insert Price fail : {0}".format(val))
    
    if isDesigner == 1:
        try:
            for (d_idx_naver, c_idx_naver, d_name, d_score, d_reservation_link, d_img_src, d_comment) in c_designer:
                sql = "INSERT INTO designer (d_idx_naver, c_idx_naver, d_name, d_score, d_reservation_link, d_img_src, d_comment) VALUES (%s,%s,%s,%s,%s,%s,%s)"
                val = (d_idx_naver, c_idx_naver, d_name, d_score, d_reservation_link, d_img_src, d_comment)
                cursor.execute(sql, val)
        except:
            print("sql-Insert designer fail : {0}".format(val))
    
    conn.commit()
conn.close()

https://m.place.naver.com/hairshop/35459030/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/35459030/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/35459030/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/35459030/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/35459030/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/35459030/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/35459030/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/35459030/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/1251281589/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1251281589/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/1536011201/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1536011201/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1521298216/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1521298216/stylist의 d_reservation_link 못찾음
https

https://m.place.naver.com/hairshop/37966596/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1676116707/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/16809625/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1320457160/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1029056393/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/1029056393/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/21491273/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/35014018/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37982177/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/19669103/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1039509281/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37979839/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/37979839/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37979839/home?entry

https://m.place.naver.com/hairshop/34013300/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1673329729/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1673329729/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/37961063/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1663498704/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1663498704/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/1976082503/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1976082503/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/21715310/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/19447831/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/19447831/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1816037984/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1816037984/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1216495072/home?ent

https://m.place.naver.com/hairshop/1403168311/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1980992684/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37965225/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37965225/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1085937007/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1085937007/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37971403/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/37971403/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37971403/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37978134/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/37978134/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1822805330/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/31487593/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37957763/home?entry

https://m.place.naver.com/hairshop/33420626/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1268812997/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/21494632/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/21494632/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37420682/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1085654663/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1767378856/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1483091117/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1483091117/stylist의 d_score 못찾음
https://m.place.naver.com/hairshop/1483091117/stylist의 d_reservation_link 못찾음
https://m.place.naver.com/hairshop/1981543435/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37983394/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/37983394/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37983394/home?entr

...

https://m.place.naver.com/hairshop/1263012851/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1263012851/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1263012851/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/37962586/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/37962586/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37962586/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37962586/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/17058867/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/17058867/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/17058867/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/36810021/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/19697716/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/19697716/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairs

https://m.place.naver.com/hairshop/1777592776/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1777592776/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1777592776/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/1851315421/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1851315421/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/37977012/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37977012/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37977012/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/1610649484/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/1610649484/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1610649484/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1610649484/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/1727430466/home?entry=pll의 c_phone 못찾음
https://

https://m.place.naver.com/hairshop/37972498/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37972498/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37972498/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/17063258/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/17063258/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/17063258/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/16795617/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/16795617/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/16795617/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/16795617/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/1719992600/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/1719992600/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1719992600/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.

https://m.place.naver.com/hairshop/37969012/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/37969012/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/37969012/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37969012/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/1799395677/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1799395677/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/68716012/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/68716012/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/16997595/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/16997595/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/16997595/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/37949537/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37949537/home?entry=pll의 c_visitor_review 못찾음
https://m.place

https://m.place.naver.com/hairshop/1168138793/home?entry=pll의 c_phone 못찾음
https://m.place.naver.com/hairshop/1168138793/home?entry=pll의 c_hours 못찾음
https://m.place.naver.com/hairshop/1168138793/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/1168138793/home?entry=pll의 c_visitor_review 못찾음
https://m.place.naver.com/hairshop/37965375/home?entry=pll의 c_score 못찾음
https://m.place.naver.com/hairshop/37965375/home?entry=pll의 c_visitor_review 못찾음
